In [ ]:
import url_fish as fish
import requests
import bs4

In [ ]:
xml_file_url = fish.get_fish_xml()

def get_spicies(xml_file):
    r = requests.get(xml_file)
    if r.status_code == 200:
        soup = bs4.BeautifulSoup(r.content, 'xml')
        spiecies_tag = soup.findAll('navn')
        return[spiecies.text for spiecies in spiecies_tag]
    
fish_labels = get_spicies(xml_file_url)

print(fish_labels)


In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv

cam = cv.VideoCapture(0)

while(cam.isOpened()):
    x, frame = cam.read()
    cv.imshow('frame', frame)
    plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv.destroyAllWindows()


In [ ]:
"""labeling data images"""
!cd ../labelImg && python labelImg.py

In [ ]:
MODEL_NAME = 'my_fish_model'
PRETRAINED_MODEL = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
import os

paths = {
'CHECKPOINT_PATH': os.path.join('fish_images','training_images','images_copy','size_100_images'),
'OUTPUT_PATH': os.path.join('fish_images','training_images','images_copy','size_100_images', 'export'),
'TFJS_PATH':os.path.join('fish_images','training_images','images_copy','size_100_images', 'tfjsexport'),
'TFLITE_PATH':os.path.join('fish_images','training_images','images_copy','size_100_images', 'tfliteexport'),
'SCRIPTS': os.path.join('Scripts'),
'ANNOTATIONS': os.path.join('Annotations'),
'PROTOC': os.path.join('Protoc'),
'PRETRAINED_MODEL_PATH':os.path.join('Pre_model_trained')
}

files = {
    'PIPELINE_CONFIG':os.path.join('fish_images','training_images','images_copy','size_100_images','pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join('Scripts','GenerateTFRecord', TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join('Annotations', LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    print(os.getcwd())
    if not os.path.exists(path):
        print(path)
        !mkdir {path}

In [ ]:
#!pip install wget
import wget

url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip {paths['PROTOC']}
!cd {paths['PROTOC']} && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC'], 'bin'))
!cd TF/Models/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd TF/Models/models/research/slim && pip install -e .

In [ ]:
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL+'.tar.gz'}

In [ ]:
labels_mapped = [{'name':'Makrel', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels_mapped:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
#python {files['TF_RECORD_SCRIPT']} -x {os.path.join('fish_images', 'training_images')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATIONS'], 'train.record')}
#!python {files['TF_RECORD_SCRIPT']} -x {os.path.join('fish_images', 'test_images')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATIONS'], 'test.record')}

from Scripts.GenerateTFRecord import generate_tfrecord as tfr

tfr.main_cath(xml_dir=os.path.join('fish_images', 'training_images', 'images_copy','size_100_images'), labels_path=files['LABELMAP'], output_path=os.path.join(paths['ANNOTATIONS'], 'train.record'))
tfr.main_cath(xml_dir=os.path.join('fish_images', 'test_images'), labels_path=files['LABELMAP'], output_path=os.path.join(paths['ANNOTATIONS'], 'test.record'))

In [ ]:
#"!copy {os.path.join('Pre_model_trained', PRETRAINED_MODEL, 'pipeline.config')} {os.path.join(path['CHECKPOINT_PATH'])}

In [ ]:
from TF.Models.models.research.object_detection.utils import config_util
from TF.Models.models.research.object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import tensorflow as tf

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
batch_size = 0
for image in os.listdir(os.path.join('fish_images','training_images','images_copy','size_100_images')):
    if image.endswith('.xml'):
        batch_size = batch_size + 1

print(batch_size)

pipeline_config.model.ssd.num_classes = 1 #TODO - len(fish_spiecies)
pipeline_config.train_config.batch_size = batch_size
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATIONS'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATIONS'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

In [ ]:
#TF_TRAINING_SCRIPT = os.path.join('TF','Models','models', 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
#steps_command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TF_TRAINING_SCRIPT, os.path.join('fish_images','training_images', 'images_copy', 'size_100_images') ,files['PIPELINE_CONFIG'], 100)

In [ ]:
#!{steps_command}

In [ ]:
#Instead of terminal-use
import TF.Models.models.research.object_detection.caths_model_tf2 as cath_model

'''Steps can be adjust'''
qnty_steps=3000
cath_model.main_model_cath(model_dir=os.path.join(paths['CHECKPOINT_PATH']),pipeline_config_path= os.path.join('fish_images','training_images','images_copy','size_100_images','pipeline.config'), train_steps=qnty_steps)

In [ ]:
#command_validation = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TF_TRAINING_SCRIPT,os.path.join('fish_images','training_images','test_model_to_cath') ,files['PIPELINE_CONFIG'], os.path.join('fish_images','training_images') )



In [ ]:
#!{command_validation}

In [ ]:
import TF.Models.models.research.object_detection.caths_model_tf2 as cath_model

cath_model.main_model_cath(model_dir=os.path.join('fish_images','training_images','images_copy', 'size_100_images'),pipeline_config_path= files['PIPELINE_CONFIG'], checkpoint_dir=os.path.join('fish_images','training_images','images_copy', 'size_100_images'))

In [ ]:
import os
import tensorflow as tf
from TF.Models.models.research.object_detection.utils import label_map_util
from TF.Models.models.research.object_detection.utils import visualization_utils as viz_utils
from TF.Models.models.research.object_detection.builders import model_builder
from TF.Models.models.research.object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
fish_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=fish_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-1')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = fish_model.preprocess(image)
    prediction_dict = fish_model.predict(image, shapes)
    detections = fish_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
import modules.prepare_images as reshape
img_100 = reshape.resizing_image_squared(os.path.join('fish_images','training_images','brugbar_makrel.jpg'),100)
img = cv2.imread(img_100)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes']+label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=.8,
    agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.8,
        agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break